In [1]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [2]:
SAMPLE_TEXT = "Does this thing really work? Lets see."

In [3]:
sent_tokenize(SAMPLE_TEXT)

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  Searched in:
    - '/home/aakash/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/home/aakash/anaconda3/nltk_data'
    - '/home/aakash/anaconda3/share/nltk_data'
    - '/home/aakash/anaconda3/lib/nltk_data'
    - ''
**********************************************************************


In [ ]:
words = word_tokenize(SAMPLE_TEXT)
words

In [ ]:
from nltk.corpus import stopwords

In [ ]:
stop = stopwords.words('english')
stop

In [ ]:
clean_words = [w for w in words if not w.lower() in stop]
clean_words

In [ ]:
stem_words = ["play", "played", "playing", "player", "happier"]
from nltk.stem import PorterStemmer
ps = PorterStemmer()
for w in stem_words:
    print (ps.stem(w))

In [ ]:
from nltk.corpus import state_union

In [ ]:
text = state_union.raw("2006-GWBush.txt")

In [ ]:
text

In [ ]:
from nltk import pos_tag

In [ ]:
from nltk import pos_tag
pos = pos_tag(word_tokenize(text.lower()))
pos[0:50]

In [ ]:
from nltk.stem import WordNetLemmatizer

In [ ]:
lem = WordNetLemmatizer()

In [ ]:
lem.lemmatize("good", pos = 'a')

In [ ]:
lem.lemmatize("better", pos = 'a')

In [ ]:
lem.lemmatize("painting", pos = 'n')

In [ ]:
lem.lemmatize("painting", pos = 'v')

In [ ]:
from nltk.corpus import wordnet

In [ ]:
w1 = wordnet.synset("ship.n.01")
w2 = wordnet.synset("boat.n.01")
w1.wup_similarity(w2)

In [ ]:
from nltk import ne_chunk

In [ ]:
sentence = "Mr. Mark Zuckerberg is the founder of the Facebook inc."
ne_chunk(pos_tag(word_tokenize(sentence.lower())))

In [ ]:
from nltk.corpus import movie_reviews
from nltk.stem import WordNetLemmatizer
lem = WordNetLemmatizer()

from nltk import pos_tag
from nltk.corpus import stopwords
import string 

stop = stopwords.words("english")
punc = list(string.punctuation)
stop = stop + punc
stop
 
def mapFromPostTagToWordnet(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''
    
def getLemmatizedWords(word_list):
    pos = pos_tag(word_list)
    lemmatized_words = []
    for (word,tag) in pos:
        tag = mapFromPostTagToWordnet(tag)
        if not (tag == ""):
            lemmatized_words.append(lem.lemmatize(word, pos = tag))
        else:
            lemmatized_words.append(word)
    return lemmatized_words

def clean_data(word_list):
    #Removing stop words
    clean_words = [w for w in word_list if not w.lower() in stop]
    #Lemmatizing
    lemmatized_words = getLemmatizedWords(clean_words)
    return lemmatized_words

In [ ]:
documents = []
for category in movie_reviews.categories():
    for fileid in movie_reviews.fileids(category):
        documents.append((clean_data(movie_reviews.words(fileid)), category))
documents[0:5]

In [ ]:
import random
random.seed(0)
random.shuffle(documents)

In [ ]:
training_documents = documents[0:1500]
testing_documents = documents[1500:]

In [ ]:
import string
all_words = []
for doc in training_documents:
    for w in doc[0]:
        all_words.append(w.lower())

In [ ]:
len(all_words)

In [ ]:
import nltk
dist = nltk.FreqDist(all_words)
features = dist.most_common(3000)
feature_words = [i[0] for i in features]

In [ ]:
import collections
def get_features(document):
    words_count = collections.Counter(document)
    words = set(document)
    features = {}
    for w in feature_words:
        if w in words:
            features[w] = words_count.get(w)
        else:
            features[w] = 0
    return features

In [ ]:
training_data = [(get_features(a), b) for (a, b) in training_documents]

In [ ]:
testing_data = [(get_features(a), b) for (a, b) in testing_documents]

In [ ]:
classifier = nltk.NaiveBayesClassifier.train(training_data)

In [ ]:
classifier.show_most_informative_features(10)

In [ ]:
nltk.classify.accuracy(classifier, testing_data)

In [ ]:
from nltk.classify.scikitlearn import SklearnClassifier
from  sklearn.svm import SVC

In [ ]:
classifier_sklearn = SklearnClassifier(SVC(random_state=0))
classifier_sklearn.train(training_data)

In [ ]:
nltk.classify.accuracy(classifier_sklearn, testing_data)

In [ ]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.ensemble import RandomForestClassifier

classifier_sklearn = SklearnClassifier(RandomForestClassifier(n_estimators=500,random_state=0))
classifier_sklearn.train(training_data)
nltk.classify.accuracy(classifier_sklearn, testing_data)